Цель проекта — построить и тонко настроить модель на основе трансформеров (RoBERTa/BERT/DeBERTa) для бинарной классификации отзывов IMDB (положительный / отрицательный).

Подход: используем datasets + transformers (Trainer) для быстрой и воспроизводимой тонкой настройки roberta-base (параметры можно легко заменить на deberta-v3-small для экспериментов). Включены: подготовка данных, токенизация, обучение с ранней остановкой, оценка, сохранение чекпоинта и экспорт предсказаний с вероятностями.

Установка зависимостей

In [ ]:
!pip install -q transformers datasets evaluate accelerate

Импорты и фиксация сидов

In [ ]:
import random
import numpy as np
import torch
from datasets import load_dataset
from transformers import (AutoTokenizer, AutoModelForSequenceClassification,
                          TrainingArguments, Trainer, EarlyStoppingCallback)
import evaluate
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import math

RND = 42
random.seed(RND)
np.random.seed(RND)
torch.manual_seed(RND)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(RND)

Загрузка датасета и разбиение на train/val/test

In [ ]:
# Загружаем imdb из HF datasets (совпадает по содержимому с Kaggle 50k)
dataset = load_dataset("imdb")
split = dataset['train'].train_test_split(test_size=0.10, seed=RND)
train_ds = split['train']
val_ds   = split['test']
test_ds  = dataset['test']

print(train_ds, val_ds, test_ds)

Токенизация и подготовка датасетов

In [ ]:
MODEL_NAME = "roberta-base"
MAX_LEN = 512

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def preprocess(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=MAX_LEN)

train_tok = train_ds.map(preprocess, batched=True, remove_columns=["text"])
val_tok   = val_ds.map(preprocess, batched=True, remove_columns=["text"])
test_tok  = test_ds.map(preprocess, batched=True, remove_columns=["text"])

# Указываем формат PyTorch
columns = ["input_ids", "attention_mask", "label"]
train_tok.set_format(type="torch", columns=columns)
val_tok.set_format(type="torch", columns=columns)
test_tok.set_format(type="torch", columns=columns)

Инициализация модели и TrainingArguments

In [ ]:
import os
# Отключаем W&B
os.environ["WANDB_DISABLED"] = "true"

from transformers import TrainingArguments, AutoModelForSequenceClassification
import torch

# Initialize the model
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

# Training args
training_args = TrainingArguments(
    output_dir="./results_imdb",
    overwrite_output_dir=True,
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=4,
    learning_rate=2e-5,
    weight_decay=0.01,
    warmup_ratio=0.06,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    logging_steps=200,
    fp16=torch.cuda.is_available(),
    seed=RND,
    # ВАЖНО: отключаем все внешние репортеры (wandb и пр.)
    report_to=["none"]
)

Trainer, метрики и само обучение

In [ ]:
from transformers import Trainer, EarlyStoppingCallback
import evaluate
import numpy as np

# Metrics
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy.compute(predictions=preds, references=labels)["accuracy"]
    f1m = f1.compute(predictions=preds, references=labels, average="weighted")["f1"]
    return {"accuracy": acc, "f1": f1m}


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tok,
    eval_dataset=val_tok,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

trainer.train()

Предсказание на тесте, отчёт, сохранение модели и экспорт CSV

In [ ]:
# Повторная инициализация Trainer перед оценкой
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tok,
    eval_dataset=val_tok, # Оставляем eval_dataset для predict, но не оцениваем явно
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)] # Убедитесь, что колбэки здесь тоже есть, если они нужны для evaluate (обычно нет)
)


# 1) Подробный отчёт и матрица ошибок на тесте
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# если ещё не получили preds (в вашем выводе уже есть pred = trainer.predict(test_tok))
pred = trainer.predict(test_tok)
logits = pred.predictions
probs = torch.nn.functional.softmax(torch.tensor(logits), dim=-1).numpy()
y_pred = np.argmax(logits, axis=1)
y_true = pred.label_ids

print("\nConfusion matrix:\n", confusion_matrix(y_true, y_pred))
print("\nClassification report:\n", classification_report(y_true, y_pred, digits=4))

# Вывод точности для всех наборов данных
# print(f"\nAccuracy on Training set: {train_metrics['eval_accuracy']:.4f}") # Убираем вывод точности на обучении
# print(f"Accuracy on Validation set: {val_metrics['eval_accuracy']:.4f}") # Убираем вывод точности на валидации
print(f"Accuracy on Test set: {(y_pred == y_true).mean():.4f}")


# 2) Сохранить чекпоинт финальной модели
trainer.save_model("./best_imdb_model")
tokenizer.save_pretrained("./best_imdb_model")

# 3) Экспорт предсказаний в CSV (для отчёта)
import pandas as pd
df = pd.DataFrame({
    "text": [test_ds[i]["text"] for i in range(len(test_ds))],
    "label": y_true,
    "pred": y_pred,
    "prob_pos": probs[:,1],
    "prob_neg": probs[:,0],
    "max_prob": probs.max(axis=1)
})
df.to_csv("imdb_test_predictions_with_probs.csv", index=False)
print("\nSaved imdb_test_predictions_with_probs.csv")

Результат выполнения задания:

Валидационная точность (validation accuracy): 0.9500 (95.00%).

Тестовая точность (test accuracy): 0.9537 (95.37%).

Precision/Recall/F1 (macro/weighted) ≈ 0.9538.